# OpenStreetMap Data Wrangling Project

### Data extraction script

* ** Program: ** Data Analyst Nanodegree 
* ** Student: ** Guillermo Naranjo
* ** Date:** August 14, 2017

Extract OSM XML data for selected geolocation, apply a basic data quality process to validate and transform nodes, ways and attribute tags to predefined entities in CSV files. 

Based on OSM Case Study, the following script reads the OSM data file specified in OSM_PATH constant and extract the tags related to Nodes, Ways and its child tags. It also apply cleaning to extracted data, trasnformation into CSV and load to SQlite database.

In [1]:
# ================================================== #
#     Importing required libraries                   #
# ================================================== #
import xml.etree.cElementTree as ET
from collections import defaultdict
import codecs
import cerberus
from unicode_dict_writer import unicode_dict_writer as udw
import schema
import re
import pprint
import sqlite3
import pandas as pd
import csv

In [2]:
# ================================================== #
#     Global constants are defined                   #
# ================================================== #
OSM_PATH = "costa_rica_greater_metropolitan_area.osm"
DB_PATH = 'osm_costa_rica.db'

DROP_QUERY = """DROP TABLE IF EXISTS """
            
CREATE_QUERY_N ="""CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);"""

CREATE_QUERY_NT = """CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);
"""
CREATE_QUERY_W = """CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);"""

CREATE_QUERY_WT = """CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);"""

CREATE_QUERY_WN = """CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);"""

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

SCHEMA = schema.schema

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\.\,\t\r\n]')
VALID_PHONE = re.compile(r'\(\d{3}\)\s*\d{8}') #using https://pythex.org/

TEXT_KEYS = {"name"}

MAPPING_CHARS = {
    ",":" de",
    ";":" o ",
    "en:":"Wikipedia ",
    "http://":"",
    "https://":"",
    "#":"número ",
    "&":"y",
    "$":"USD",
    ">":"",
    "<":"",
    "'":"",
    '"':"",
    "*":"",
    "+":"más",
    "%E9":"é",
    "%20":" ",
    "Desamaprados":"Desamparados",
    "Abast.":"Abastecedor",
    "Also knows as":"Conocido como",
    "Av.":"Avenida",
    "4ta": "cuarta",
    "Jr.":"Junior",
    "No.":"número ",
    "Dr.":"Doctor",
    "Dr":"Doctor",    
    "Dra.":"Doctora",
    "Dra":"Doctora",
    "S.A.":"SA",
    "Hnas.":"Hermanas",
    "Hnos.":"Hermanos",
    "MSc.":"Máster",    
    "Intl.":"Internacional",
    "!":"",
    "R.L.":"RL",
    "Urb.":"Urbanización",
    "Mo-Sa":"Lu-Sa",
    "Mo-Fr":"Lu-Vi",
    "Mo-Su":"Lu-Do",
    " Su ":" Do ",
    "Mo-Th":"Lu-Ju",
    "S.A:":"SA"
}

MAPPING = {
    "AM-PM":"AMPM",
    "Aeroscasillas":"Aerocasillas",
    "Azafran":"Azafrán",
    "Cerrajeria":"Cerrajería",
    "Coopeserviodores":"Coopeservidores",
    "JetBox":"Jetbox",
    "Macrobiotica":"Macrobiótica",
    "MegaSuper":"Megasuper",
    "Metropolis":"Metrópolis",
    "Metropoli":"Metrópolis",
    "Musammani":"Musmanni",
    "Musmani":"Musmanni",
    "Musmanny":"Musmanni",
    "Mussmani":"Musmanni",
    "Muswanni":"Musmanni",
    "Panaderia":"Panadería",
    "Pizzeria":"Pizzería",
    "Pulperia":"Pulpería",
    "Quiros":"Quirós",
    "Veterniaria":"Veterinaria",
    "Cartago1":"Cartago",
    "Rohmoser":"Rohrmoser"
}

In [3]:
# ================================================== #
#               Validity Functions                   #
# ================================================== #

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema """
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

def is_problematic(kvalue):
    """ Validates if key value contains problematic characters based on PROBLEMCHARS regex """
    return PROBLEMCHARS.search(kvalue)  

def insert_text_separator(text):
    """ Transforms text values to UTF-8 """
    return text.encode('utf-8')

def right_type(key,value):
    """ Base on the item list, set the correct value datatype """
    if key in ['id','uid','changeset','version']:
        return int(value)
    elif key in ['lat','lon']:
        return float(value)
    else:
        return value
    
def log(id,key,value,newvalue):
    
    ''' Show estandar log message for changes applied '''
    
    print "TAG",id," [",key.upper(),",", value, "] fixed to [",key.upper(),",", newvalue.decode('utf-8'),"]"   

In [4]:
# ================================================== #
#               Parser Functions                     #
# ================================================== #

def get_element(osm_file, tags=('node', 'way', 'relation')):
    
    """ Parse xml tree and filter to only include node, ways and relation nodes """
    
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def get_key_type(k,default_tag_type='regular'):    
    
    """ Parse attribute k names to set the correct name and type if : exists in the name """
    
    klist = k.split(':')
    newkey = newtype = ""
    if len(klist) > 2:
        newkey = klist[1] + ":" + klist[2]
        newtype = klist[0]
    elif len(klist) == 2:
        newkey = klist[1]
        newtype = klist[0]
    else:
        newkey = klist[0]
        newtype = default_tag_type
    return newkey,newtype

def get_tags(_id, elements):
    
    """ get list of child tags dictionaries from node or ways tags """
    
    tags = []
    for tag in elements.iter("tag"):        
        if 'k' in tag.attrib:
            _key,_type = get_key_type(tag.attrib['k'])
            _value = tag.attrib['v'] 
            dtag = {}
            dtag['id'] = int(_id) 
            dtag['key'] = insert_text_separator(_key)
            dtag['value'] = insert_text_separator(_value)
            dtag['type'] = insert_text_separator(_type)
            tags.append(dtag)                   
    return tags

def get_nodes(id, elements):
    
    """ get list of child nodes dictionaries from ways tags """
    
    nodes = []
    i = 0
    for node in elements.iter("nd"):                
        dnode = {}
        dnode['id'] = int(id) 
        dnode['node_id'] = int(node.attrib['ref'])
        dnode['position'] = int(i) 
        nodes.append(dnode)        
        i+=1
    return nodes  

In [5]:
# ================================================== #
#           Postal code cleaning functions         #
# ================================================== #

def fix_postcode(dic):
    
    """ returns the correct key value and type """
    
    dic['key'] = 'postcode'
    dic['type'] = 'addr'
    log(dic['id'],'postal_code','postal_code','postcode')

In [6]:
# ================================================== #
#           Phone Numbers Cleaning Functions         #
# ================================================== #

def is_valid_phone(phone):
    
    """ check if phone comply with REGEX specification """
    
    return  VALID_PHONE.search(phone) is not None

def fix_phone(dic):
    
    """ update phone numbers in order to use expected format """
    
    phone = origphone = dic['value']
    # taken from: https://stackoverflow.com/questions/5658369/how-to-input-a-regex-in-string-replace 
    phone = re.sub(r"-|\+|,|\ |\(|\)|\ *","", phone).replace("tel:","")
    if len(phone) == 11:
        phone = "(506)" + phone[-8:]
    elif len(phone) == 8:
        phone = "(506)" + phone    
    dic['value'] = phone    
    log(dic['id'],dic['key'],origphone,dic['value'])

In [7]:
def fix_chars(_value, chars=MAPPING_CHARS):   
    
    ''' Replacing multiple character with value ones'''
    
    #Taken from https://stackoverflow.com/questions/6116978/python-replace-multiple-strings
    
    chars = dict((re.escape(k), v) for k, v in chars.iteritems())
    pattern = re.compile("|".join(chars.keys()))
    text = pattern.sub(lambda m: chars[re.escape(m.group(0))], _value)
    return text

In [8]:
# ================================================== #
#               Tag Names Cleaning Functions         #
# ================================================== #

def fix_name(dic, mapping=MAPPING):
    
    """ check tag name values against expected set, updating those with the expected value as required """    
    _value = dic['value']
    _key = dic['key']
    _type = dic['type']
    my_map = pd.Series(mapping)    

    if  _key in ['payment','network']: #special cases were commas are valid separator
        dic['value'] = _value.replace(","," o ")
        log(dic['id'],_key,_value,dic['value'])        
    elif _value in my_map:  #this replaces invalid places names with correct ones
        dic['value'] = my_map[_value]
        log(dic['id'],'name',_value,dic['value'])   
    elif is_problematic(_value): #this replaces invalid characters in values with valid ones
        dic['value'] = fix_chars(_value)
        log(dic['id'],_key,_value,dic['value'])        

In [9]:
# ================================================== #
#          Master Cleaning Functions         #
# ================================================== #

def clean_tags(tags, textkeys=TEXT_KEYS):
    
    """ clean tags by fixing phone numbers, postal codes and names """
    
    for dic in tags:
        if dic['key'] == 'postal_code':            
            fix_postcode(dic)
        elif (dic['key'] == 'phone' or dic['key'] == 'fax') and not is_valid_phone(dic['value']):
            fix_phone(dic)
        else:
            fix_name(dic)

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS):

    """ creates main dictionary structure based on OSM xml extract and clean as required """
    
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []

    if element.tag == 'node':        
        
        for item in node_attr_fields:
            node_attribs[item] = right_type(item,element.attrib[item])
            
        tags = get_tags(node_attribs['id'],element)
        
        #This script apply cleaning process on nodes tags
        clean_tags(tags)
        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        
        for item in way_attr_fields:
            way_attribs[item] = right_type(item,element.attrib[item])
            
        way_nodes = get_nodes(way_attribs['id'],element)                        
        tags = get_tags(way_attribs['id'],element)
        
        #This script apply cleaning process on way tags
        clean_tags(tags)

        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

In [10]:
# ================================================== #
#       Database Creation and load Functions         #
# ================================================== #

def fill_table(table,conn):
    
    '''I used tentacles666 code to simplify data import. 
       https://tentacles666.wordpress.com/2014/11/14/python-creating-a-sqlite3-database-from-csv-files/'''
    
    conn.text_factory = str  # allows utf-8 data to be stored   
    curs = conn.cursor() 
    curs.execute('PRAGMA encoding="UTF-8";')
    
    # traverse the directory and process each .csv file    
    csvfile = table + ".csv"
    with open(csvfile, "rb") as f:
        reader = csv.reader(f)
 
        header = True
        for row in reader:
            if header:
                # gather column names from the first row of the csv
                header = False
                insertsql = "INSERT INTO %s VALUES (%s)" % (table,
                            ", ".join([ "?" for column in row ]))
 
                rowlen = len(row)
            else:
                # skip lines that don't have the right number of columns
                if len(row) == rowlen:
                    curs.execute(insertsql, row)   
                    
def fill_tables(conn):
    
    """ import csv to tables """
    
    curs = conn.cursor()
    fill_table("nodes",conn)
    fill_table("nodes_tags",conn)
    fill_table("ways",conn)
    fill_table("ways_tags",conn)
    fill_table("ways_nodes",conn)
    conn.commit()  
    curs.close()
    conn.close()
                    
def drop_tables(conn):
    
    """ drop tables is exist """
    
    curs = conn.cursor()
    curs.execute(DROP_QUERY + "nodes")
    curs.execute(DROP_QUERY + "nodes_tags")
    curs.execute(DROP_QUERY + "ways")
    curs.execute(DROP_QUERY + "ways_tags")
    curs.execute(DROP_QUERY + "ways_nodes")    
    conn.commit()   

def create_tables(conn):
    
    """ create tables """
    
    curs = conn.cursor()
    curs.execute(CREATE_QUERY_N)
    curs.execute(CREATE_QUERY_NT)
    curs.execute(CREATE_QUERY_W)
    curs.execute(CREATE_QUERY_WT)
    curs.execute(CREATE_QUERY_WN)
    conn.commit()

In [11]:
# ================================================== #
#         Master creation and load DB Functions      #
# ================================================== #
def process_db(db,drop=DROP_QUERY):
    
    """ create osm database and import previously created csv """
    
    conn = sqlite3.connect(db)
    drop_tables(conn)
    create_tables(conn)
    fill_tables(conn)
    conn.close()
    print " DB creation and csv data import successful."

In [12]:
# ================================================== #
#       Main Function to Generate CSV                #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'wb') as nodes_file, \
        codecs.open(NODE_TAGS_PATH, 'wb') as nodes_tags_file, \
        codecs.open(WAYS_PATH, 'wb') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'wb') as way_nodes_file, \
        codecs.open(WAY_TAGS_PATH, 'wb') as way_tags_file:

        nodes_writer = udw(nodes_file, NODE_FIELDS)
        node_tags_writer = udw(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = udw(ways_file, WAY_FIELDS)
        way_nodes_writer = udw(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = udw(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        i = 0
        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                i+=1
                if validate is True:
                    validate_element(el, validator)
                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])
                if i % 2000 == 1: print "\n", i-1,"TAGS PROCESSED. \n"
    print " OSM XML extraction, cleanup and csv creation successful."

In [13]:
if __name__ == "__main__":
    process_map(OSM_PATH, validate=True)    
    process_db(DB_PATH)


0 TAGS PROCESSED. 

TAG 114940955  [ EXIT_TO , 111, Plaza Real Cariari ] fixed to [ EXIT_TO , 111 de Plaza Real Cariari ]
TAG 115641809  [ EXIT_TO , Escazú; CN 105; CN 167 ] fixed to [ EXIT_TO , Escazú o  CN 105 o  CN 167 ]
TAG 167733585  [ EXIT_TO , Santa Ana; CN 147 ] fixed to [ EXIT_TO , Santa Ana o  CN 147 ]
TAG 167737300  [ EXIT_TO , Santa Ana; CN 147 ] fixed to [ EXIT_TO , Santa Ana o  CN 147 ]

2000 TAGS PROCESSED. 

TAG 197698100  [ BN , স্যান হোসে, কোস্টা রিকা ] fixed to [ BN , স্যান হোসে de কোস্টা রিকা ]
TAG 197698100  [ BS , San Jose, Kostarika ] fixed to [ BS , San Jose de Kostarika ]
TAG 197698100  [ GL , San Xosé, Costa Rica ] fixed to [ GL , San Xosé de Costa Rica ]
TAG 197698100  [ MR , सान होजे, कोस्टा रिका ] fixed to [ MR , सान होजे de कोस्टा रिका ]
TAG 197698100  [ SV , San José, Costa Rica ] fixed to [ SV , San José de Costa Rica ]
TAG 197698100  [ TA , சான் ஹொசே, கோஸ்ட்டா ரிக்கா ] fixed to [ TA , சான் ஹொசே de கோஸ்ட்டா ரிக்கா ]
TAG 197698100  [ WIKIPEDIA , en:San J

TAG 320780716  [ CREATED_BY , Merkaartor 0.12 ] fixed to [ CREATED_BY , Merkaartor 0.12 ]
TAG 330369451  [ CREATED_BY , Potlatch 0.10f ] fixed to [ CREATED_BY , Potlatch 0.10f ]
TAG 330369452  [ CREATED_BY , Potlatch 0.10f ] fixed to [ CREATED_BY , Potlatch 0.10f ]
TAG 331104211  [ IS_IN , Aserrí, San José ] fixed to [ IS_IN , Aserrí de San José ]
TAG 331495305  [ CREATED_BY , Potlatch 0.10f ] fixed to [ CREATED_BY , Potlatch 0.10f ]
TAG 331530001  [ CREATED_BY , Merkaartor 0.12 ] fixed to [ CREATED_BY , Merkaartor 0.12 ]

26000 TAGS PROCESSED. 

TAG 332230261  [ CREATED_BY , Potlatch 0.10f ] fixed to [ CREATED_BY , Potlatch 0.10f ]
TAG 332232192  [ CREATED_BY , Potlatch 0.10f ] fixed to [ CREATED_BY , Potlatch 0.10f ]
TAG 333089476  [ CREATED_BY , Merkaartor 0.12 ] fixed to [ CREATED_BY , Merkaartor 0.12 ]
TAG 333089477  [ CREATED_BY , Merkaartor 0.12 ] fixed to [ CREATED_BY , Merkaartor 0.12 ]
TAG 333089479  [ CREATED_BY , Merkaartor 0.12 ] fixed to [ CREATED_BY , Merkaartor 0.12 ]
T

TAG 444630625  [ EXIT_TO , Radial Coyol, Riteve ] fixed to [ EXIT_TO , Radial Coyol de Riteve ]
TAG 445164738  [ NAME , Parada de bus San Isidro-Heredia, San Rafael-Heredia ] fixed to [ NAME , Parada de bus San Isidro-Heredia de San Rafael-Heredia ]
TAG 445197843  [ PHONE , +506 2237-2105 ] fixed to [ PHONE , (506)22372105 ]
TAG 445517925  [ IS_IN , Alajuela, Costa Rica ] fixed to [ IS_IN , Alajuela de Costa Rica ]
TAG 446464241  [ CITY , Hatillo #6 ] fixed to [ CITY , Hatillo número 6 ]
TAG 446464241  [ NAME , Mas X Menos Hatillo #3022 ] fixed to [ NAME , Mas X Menos Hatillo número 3022 ]
TAG 446464246  [ NAME , Más x Menos Plaza America #3034 ] fixed to [ NAME , Más x Menos Plaza America número 3034 ]
TAG 446464246  [ PHONE , 2254-5458 / 2254-6995 ] fixed to [ PHONE , 22545458/22546995 ]
TAG 446464246  [ WEBSITE , http://www.walmartmexico.com.mx/directorio_costarica.html ] fixed to [ WEBSITE , www.walmartmexico.com.mx/directorio_costarica.html ]
TAG 448805075  [ IS_IN , Desamparados,

TAG 813106984  [ NAME , Rest. Hacienda Rocio ] fixed to [ NAME , Rest. Hacienda Rocio ]
TAG 821910536  [ FAX , 2296-7272 ] fixed to [ FAX , (506)22967272 ]
TAG 821910536  [ NAME , Comercial de Seguros S.A. ] fixed to [ NAME , Comercial de Seguros SA ]
TAG 821910536  [ PHONE , 2296-8282 ] fixed to [ PHONE , (506)22968282 ]
TAG 821910536  [ WEBSITE , http://www.comerseguros.com ] fixed to [ WEBSITE , www.comerseguros.com ]
TAG 822371472  [ NAME , B.C.A.C. ] fixed to [ NAME , B.C.A.C. ]
TAG 822444249  [ NAME , Pulp. Vista Azul ] fixed to [ NAME , Pulp. Vista Azul ]
TAG 822467737  [ ALT_NAME , Banco General de Costa Rica, Plaza Freses Curridabat ] fixed to [ ALT_NAME , Banco General de Costa Rica de Plaza Freses Curridabat ]
TAG 822467737  [ OPENING_HOURS , Mo-Fr 09:00-18:00; Sa 09:00-12:30 ] fixed to [ OPENING_HOURS , Lu-Vi 09:00-18:00 o  Sa 09:00-12:30 ]
TAG 822467737  [ PHONE , (506) 2588-4610 ] fixed to [ PHONE , (506)25884610 ]

56000 TAGS PROCESSED. 

TAG 825394458  [ LOCATION , Mult

TAG 1235698997  [ NAME , Parada H. México ] fixed to [ NAME , Parada H. México ]

72000 TAGS PROCESSED. 

TAG 1247865723  [ NAME , Soda Lety's ] fixed to [ NAME , Soda Letys ]
TAG 1256006240  [ NAME , McDonald's ] fixed to [ NAME , McDonalds ]
TAG 1256869908  [ NAME , Bar & Restaurante el Ruisenor ] fixed to [ NAME , Bar y Restaurante el Ruisenor ]
TAG 1256877985  [ NAME , Condominio & Residencial La Montaña ] fixed to [ NAME , Condominio y Residencial La Montaña ]
TAG 1256898669  [ NAME , Pali San Isidro de Heredia #4084 ] fixed to [ NAME , Pali San Isidro de Heredia número 4084 ]
TAG 1262969182  [ NAME , Cruces, San Pablo de Heredia ] fixed to [ NAME , Cruces de San Pablo de Heredia ]
TAG 1262971441  [ PHONE , 506 2238 4149 ] fixed to [ PHONE , (506)22384149 ]
TAG 1262982363  [ NAME , MegaSuper ] fixed to [ NAME , Megasuper ]

74000 TAGS PROCESSED. 

TAG 1268292069  [ NAME , Mora's Bar ] fixed to [ NAME , Moras Bar ]
TAG 1274593545  [ IS_IN , Flores, Heredia ] fixed to [ IS_IN , Flor

...

TAG 394753376  [ NAME , Yakky's ] fixed to [ NAME , Yakkys ]
TAG 395163574  [ PHONE , 2280-9952 ] fixed to [ PHONE , (506)22809952 ]
TAG 395347479  [ NAME , La Union S.A. ] fixed to [ NAME , La Union SA ]
TAG 395347483  [ NAME , Consultorio Fisioterapia Lic. Gerson Aguilar ] fixed to [ NAME , Consultorio Fisioterapia Lic. Gerson Aguilar ]
TAG 395361289  [ NAME , Tito's Bar ] fixed to [ NAME , Titos Bar ]
TAG 395361291  [ NAME , Rest. Las Brumas ] fixed to [ NAME , Rest. Las Brumas ]

346000 TAGS PROCESSED. 

TAG 395361616  [ NOTE , NO MODIFICAR. La jerarquía de vías nacionales está establecida por ley sin importar material de la calzada o volumen de tránsito. Primarias de 1 a 99 (excluye las autopistas), secundarias de 100 a 299 y terciarias de 300 a 999. ] fixed to [ NOTE , NO MODIFICAR. La jerarquía de vías nacionales está establecida por ley sin importar material de la calzada o volumen de tránsito. Primarias de 1 a 99 (excluye las autopistas) de secundarias de 100 a 299 y terciaria

TAG 406988463  [ NOTE , Autopista Braulio Carrillo -- ref=32 ] fixed to [ NOTE , Autopista Braulio Carrillo -- ref=32 ]
TAG 406988466  [ NOTE , Autopista Braulio Carrillo -- ref=32 ] fixed to [ NOTE , Autopista Braulio Carrillo -- ref=32 ]
TAG 406994073  [ NAME , Radial Francisco J. Orlich ] fixed to [ NAME , Radial Francisco J. Orlich ]
TAG 408023565  [ NAME , Radial Francisco J. Orlich ] fixed to [ NAME , Radial Francisco J. Orlich ]
TAG 408342492  [ NAME , Ruta #203 ] fixed to [ NAME , Ruta número 203 ]
TAG 408342548  [ NAME , Ruta #203 ] fixed to [ NAME , Ruta número 203 ]
TAG 408342549  [ NAME , Ruta #203 ] fixed to [ NAME , Ruta número 203 ]
TAG 408343229  [ NAME , Ruta #203 ] fixed to [ NAME , Ruta número 203 ]
TAG 408343291  [ NAME , Ruta #203 ] fixed to [ NAME , Ruta número 203 ]
TAG 408343292  [ NAME , Ruta #203 ] fixed to [ NAME , Ruta número 203 ]
TAG 408343740  [ NAME , Ruta #203 ] fixed to [ NAME , Ruta número 203 ]
TAG 408343741  [ NAME , Ruta #203 ] fixed to [ NAME , Ru

TAG 414277852  [ NAME , Museo Histórico Dr. Rafael Ángel Calderón Guardia ] fixed to [ NAME , Museo Histórico Doctor Rafael Ángel Calderón Guardia ]
TAG 414277852  [ PHONE , 2221-1239 / 2257-2396 ] fixed to [ PHONE , 22211239/22572396 ]
TAG 415077527  [ PHONE , +506 22924691 ] fixed to [ PHONE , (506)22924691 ]
TAG 416223399  [ WEBSITE , www.rc.cr ] fixed to [ WEBSITE , www.rc.cr ]
TAG 416225152  [ WEBSITE , www.rc.cr ] fixed to [ WEBSITE , www.rc.cr ]
TAG 417428636  [ NOTE , Please note that this part of the street is NOT a trunk. Trunks are defined here: http://wiki.openstreetmap.org/wiki/Tag:highway%3Dtrunk ] fixed to [ NOTE , Please note that this part of the street is NOT a trunk. Trunks are defined here: wiki.openstreetmap.org/wiki/Tag:highway%3Dtrunk ]
TAG 417981177  [ NOTE , Please%20note%20that%20this%20part%20of%20the%20street%20is%20NOT%20a%20trunk.%20Trunks%20are%20defined%20here:%20http://wiki.openstreetmap.org/wiki/Tag:highway%3Dtrunk ] fixed to [ NOTE , Please note that 

TAG 451502171  [ NAME , Autolavado La Colina J&R ] fixed to [ NAME , Autolavado La Colina JyR ]

350000 TAGS PROCESSED. 

TAG 451987085  [ NAME , Coopesalud R.L. - Rincón Grande ] fixed to [ NAME , Coopesalud RL - Rincón Grande ]
TAG 452566075  [ NAME , Laboratorio Nal. de Aguas ] fixed to [ NAME , Laboratorio Nal. de Aguas ]
TAG 452583109  [ KEY , only one way;two way ] fixed to [ KEY , only one way o two way ]
TAG 452603936  [ NAME , C/ Chirivico ] fixed to [ NAME , C/ Chirivico ]
TAG 452603973  [ NAME , C/ Dimosa ] fixed to [ NAME , C/ Dimosa ]
TAG 453116279  [ NOTE , NO MODIFICAR. La jerarquía de vías nacionales está establecida por ley sin importar material de la calzada o volumen de tránsito. Primarias de 1 a 99 (excluye las autopistas), secundarias de 100 a 299 y terciarias de 300 a 999. ] fixed to [ NOTE , NO MODIFICAR. La jerarquía de vías nacionales está establecida por ley sin importar material de la calzada o volumen de tránsito. Primarias de 1 a 99 (excluye las autopistas)

TAG 481427452  [ PHONE , +506 22391049 ] fixed to [ PHONE , (506)22391049 ]
TAG 481841320  [ OPERATOR , Auto Mercado S.A. ] fixed to [ OPERATOR , Auto Mercado SA ]
TAG 482109682  [ NAME , Zacate Ornamental y Decorativo S.A: ] fixed to [ NAME , Zacate Ornamental y Decorativo SA ]
TAG 488982631  [ NAME , Hospital Dr. Maximiliano Peralta Jiménez - Nuevo ] fixed to [ NAME , Hospital Doctor Maximiliano Peralta Jiménez - Nuevo ]
TAG 489126918  [ NAME , Centro de Desarrollo Social  Dr. Guillermo Padilla Castro ] fixed to [ NAME , Centro de Desarrollo Social  Doctor Guillermo Padilla Castro ]
TAG 490214768  [ NAME , Parquecito de Bellas Artes, Universidad de Costa Rica ] fixed to [ NAME , Parquecito de Bellas Artes de Universidad de Costa Rica ]
TAG 490214769  [ NAME , Parque de la Facultad de Educación, Univeridad de Costa Rica ] fixed to [ NAME , Parque de la Facultad de Educación de Univeridad de Costa Rica ]
TAG 490214770  [ WEBSITE , http://www.estudiosgenerales.ucr.ac.cr/ ] fixed to [ WE